In [1]:
!pip install lambeq
!pip install discopy
!pip install rouge-score
!pip install pytket pytket-qiskit
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.3/364.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=467e469e93945db7bc472eb5c299ff3a8ab94f63e38418afa8c215fdb8adeba6
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import statistics
from pytket import Qubit
from rouge_score import rouge_scorer
from pytket.pauli import Pauli, QubitPauliString
from pytket.utils.operators import QubitPauliOperator
from lambeq import AtomicType, BobcatParser, IQPAnsatz
from nltk.translate.bleu_score import modified_precision
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.qiskit import AerBackend, tk_to_qiskit

In [3]:
lambeqParser = BobcatParser()
N = AtomicType.NOUN
S = AtomicType.SENTENCE
quantumAnsatz = IQPAnsatz({N: 1, S: 1}, n_layers=2)
quantumBackend = AerBackend()
parameterValue = 0.25

Evaluating checksum: 0.0%|          |0.000/1.533GB [00:00<?]

Extracting model...


In [4]:
def generateQuantumCircuit(sentence, draw):
  diagram = lambeqParser.sentence2diagram(sentence)
  circuit = quantumAnsatz(diagram)
  tketCircuit = circuit.to_tk()
  parameters = dict.fromkeys(tketCircuit.free_symbols(), parameterValue)
  tketCircuit.symbol_substitution(parameters)
  compiledCircuit = quantumBackend.get_compiled_circuit(tketCircuit)
  if draw:
    diagram.draw()
    render_circuit_jupyter(tketCircuit)
  return compiledCircuit

def generateZObservable(circuit):
  n = circuit.n_qubits
  observableString = QubitPauliString({Qubit(i): Pauli.Z for i in range(n)})
  observable = QubitPauliOperator({observableString: 1})
  return observable

def similarity(exp1, exp2):
  distance = abs(exp1 - exp2)
  similarity = 1 / (1 + distance)
  return similarity

In [5]:
def BLEU(target, hypothesis, n=1):
  scores = []
  for i in range(len(target.split())):
    scores.append(float(modified_precision(target.split(' ')[i], hypothesis.split(' ')[i], n)))
  return statistics.mean(scores)

def ROUGE(target, hypothesis, n="L"):
  scorer = rouge_scorer.RougeScorer([f'rouge{n}'])
  scores = []
  for i in range(len(target.split())):
    scores.append(scorer.score(target.split()[i], hypothesis.split()[i])[f'rouge{n}'].precision)
  return statistics.mean(scores)

def LAMBEQ(target, hypothesis, draw=False):
  tCirc = generateQuantumCircuit(target, draw)
  hCirc = generateQuantumCircuit(hypothesis, draw)
  tOp = generateZObservable(tCirc)
  hOp = generateZObservable(hCirc)
  tExp = quantumBackend.get_operator_expectation_value(tCirc, tOp)
  hExp = quantumBackend.get_operator_expectation_value(hCirc, hOp)
  return similarity(tExp, hExp)

In [6]:
statements = ['Your action was a success.', 'Your action was very successful.', 'Your action was barely successful.', 'Your action was barely unsuccessful.', 'Your action was very unsuccessful.', 'Your action was a failure.']

In [7]:
for i, firstStatement in enumerate(statements):
  for secondStatement in statements[i:]:
    print(firstStatement, secondStatement)
    print('BLEU:\t', BLEU(firstStatement, secondStatement))
    print('ROUGE:\t', ROUGE(firstStatement, secondStatement))
    print('LAMBEQ:\t', LAMBEQ(firstStatement, secondStatement))
    print()

Your action was a success. Your action was a success.
BLEU:	 0.925
ROUGE:	 1.0
LAMBEQ:	 0.9986442444449937

Your action was a success. Your action was very successful.
BLEU:	 0.6909090909090909
ROUGE:	 0.6
LAMBEQ:	 0.9901792728031081

Your action was a success. Your action was barely successful.
BLEU:	 0.7242424242424242
ROUGE:	 0.6
LAMBEQ:	 0.996379709010122

Your action was a success. Your action was barely unsuccessful.
BLEU:	 0.7102564102564103
ROUGE:	 0.6
LAMBEQ:	 0.9942230711976335

Your action was a success. Your action was very unsuccessful.
BLEU:	 0.676923076923077
ROUGE:	 0.6
LAMBEQ:	 0.9957396255564013

Your action was a success. Your action was a failure.
BLEU:	 0.875
ROUGE:	 0.8
LAMBEQ:	 0.984993104782902

Your action was very successful. Your action was very successful.
BLEU:	 0.9272727272727272
ROUGE:	 1.0
LAMBEQ:	 0.99147963518211

Your action was very successful. Your action was barely successful.
BLEU:	 0.8272727272727273
ROUGE:	 0.8
LAMBEQ:	 0.99219241380973

Your ac